In [ ]:
import pandas as pd
import re

# Sample data
df = pd.DataFrame({
    'raw_date': ['Mon, 28th May 2020 12:34:56 GMT', 'Tue, 1st Jan 2019 05:00:00 GMT']
})

# Remove day suffixes like 'st', 'nd', 'rd', 'th' using regex
df['cleaned_date'] = df['raw_date'].apply(lambda x: re.sub(r'(\d+)(st|nd|rd|th)', r'\1', x))

# Parse into datetime format
df['parsed_date'] = pd.to_datetime(df['cleaned_date'], format='%a, %d %b %Y %H:%M:%S %Z', errors='coerce')

# Optional: just keep the date part (no time)
df['just_date'] = df['parsed_date'].dt.date

print(df[['raw_date', 'just_date']])


In [ ]:
import matplotlib.pyplot as plt

# Sample data
years = ['2010', '2025']
car1_prices = [15000, 18000]
car2_prices = [20000, 25000]
car3_prices = [17000, 22000]

# Plot
fig, ax = plt.subplots()

# Stacked bars
bar1 = ax.bar(years, car1_prices, label='Car 1')
bar2 = ax.bar(years, car2_prices, bottom=car1_prices, label='Car 2')

# Compute bottoms for car 3 (stacked on car1 + car2)
car1_plus_car2 = [c1 + c2 for c1, c2 in zip(car1_prices, car2_prices)]
bar3 = ax.bar(years, car3_prices, bottom=car1_plus_car2, label='Car 3')

# Add labels and legend
ax.set_ylabel('Total Price ($)')
ax.set_title('Stacked Car Prices in 2010 vs 2025')
ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
import shap
import seaborn as sns
import matplotlib.pyplot as plt

# Get SHAP interaction values
explainer = shap.TreeExplainer(model)
shap_interaction_values = explainer.shap_interaction_values(X)  # shape: [n_samples, n_features, n_features]

# Define feature of interest
feature_name = 'A'
feature_index = list(X.columns).index(feature_name)

# Interaction values of 'A' with every other feature (averaged across samples)
interaction_strengths = np.abs(shap_interaction_values[:, feature_index, :]).mean(axis=0)

# Create DataFrame for plotting
interaction_df = pd.DataFrame({
    'feature': X.columns,
    'interaction_strength': interaction_strengths
}).sort_values(by='interaction_strength', ascending=False)

# Optional: Drop self-interaction if you only want cross-feature effects
interaction_df = interaction_df[interaction_df.feature != feature_name]

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(data=interaction_df, x='interaction_strength', y='feature')
plt.title(f"Mean SHAP Interaction Strengths with '{feature_name}'")
plt.xlabel("Mean |SHAP Interaction Value|")
plt.ylabel("Interacting Feature")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
CMAKE_ARGS="-DLLAMA_CUDA=OFF -DLLAMA_METAL=OFF -DLLAMA_CLBLAST=OFF -DLLAMA_BLAS=OFF" \
pip install --force-reinstall --no-cache-dir --no-binary llama-cpp-python llama-cpp-python

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore

# --- Step 1: Simulate weekly purchase data ---
np.random.seed(0)
weeks = pd.date_range("2023-01-01", periods=52, freq='W')
states = ['Texas', 'Oklahoma', 'Louisiana', 'California', 'New York']

def make_trend(base, noise=5):
    return base + np.random.normal(0, noise, size=len(weeks))

data = {
    'Texas': make_trend(np.linspace(100, 200, 52)),
    'Oklahoma': make_trend(np.linspace(90, 180, 52)),
    'Louisiana': make_trend(np.linspace(105, 205, 52)),
    'California': make_trend(np.linspace(300, 400, 52)),
    'New York': make_trend(np.linspace(200, 100, 52)),
}

df = pd.DataFrame(data, index=weeks)

# --- Step 2: Normalize trends (Z-score) ---
df_normalized = df.apply(zscore)

# --- Step 3: Hyper-tune K using silhouette score ---
X = df_normalized.T  # states as rows
sil_scores = {}
for k in range(2, min(len(X), 10)):
    model = KMeans(n_clusters=k, random_state=0)
    labels = model.fit_predict(X)
    score = silhouette_score(X, labels)
    sil_scores[k] = score

best_k = max(sil_scores, key=sil_scores.get)
print(f"Best number of clusters: {best_k} with silhouette score: {sil_scores[best_k]:.3f}")

# --- Step 4: Fit final model and visualize ---
final_model = KMeans(n_clusters=best_k, random_state=0)
labels = final_model.fit_predict(X)
cluster_df = pd.DataFrame({'state': X.index, 'cluster': labels})
print(cluster_df)

# --- Plot silhouette scores ---
plt.figure(figsize=(8, 5))
plt.plot(list(sil_scores.keys()), list(sil_scores.values()), marker='o')
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score vs. Number of Clusters")
plt.grid(True)
plt.xticks(range(2, min(len(X), 10)))
plt.axvline(x=best_k, linestyle='--', color='red', label=f'Best k = {best_k}')
plt.legend()
plt.tight_layout()
plt.show()


# --- Step 5: Plot each cluster's normalized time trends ---
for cluster in cluster_df['cluster'].unique():
    cluster_states = cluster_df[cluster_df['cluster'] == cluster]['state']
    df_normalized[cluster_states].plot(title=f"Cluster {cluster} Trends")
    plt.ylabel("Z-Score Normalized Purchases")
    plt.show()

# --- Step 6: Correlation heatmap ---
corr = df.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title("Raw Correlation of Purchase Trends")
plt.show()

# --- Step 7: Intra-cluster correlation check ---
def mean_cluster_corr(corr_matrix, cluster_df):
    results = {}
    for cluster in cluster_df['cluster'].unique():
        members = cluster_df[cluster_df['cluster'] == cluster]['state']
        intra = corr_matrix.loc[members, members].values
        intra_corr = intra[np.triu_indices_from(intra, k=1)]
        results[f"Cluster {cluster} mean corr"] = np.mean(intra_corr)
    return results

print(mean_cluster_corr(corr, cluster_df))


In [ ]:
# Convert dates to datetime if they aren't already
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date']   = pd.to_datetime(df['end_date'])

# 1️⃣ Recode: 1 if end_date is after start_date, else 0
df['date_after_flag'] = (df['end_date'] > df['start_date']).astype(int)

# 2️⃣ Recode: 1 if 'comments' contains 'hi' (case-insensitive), else 0
df['contains_hi_flag'] = df['comments'].str.contains('hi', case=False, na=False).astype(int)

# Sort so earliest date comes first
df = df.sort_values(by=['email', 'date'])

# Drop duplicates, keeping first occurrence (earliest date)
df_first_date = df.drop_duplicates(subset='email', keep='first')

# 1️⃣ Boolean masks
mask_saiid = df['campaign'].str.contains('saiid', case=False, na=False)
mask_not_saiid = ~mask_saiid

# 2️⃣ Sets of emails
emails_saiid = set(df.loc[mask_saiid, 'email'])
emails_not_saiid = set(df.loc[mask_not_saiid, 'email'])

# 3️⃣ Intersection and differences
emails_both = emails_saiid & emails_not_saiid
emails_only_saiid = emails_saiid - emails_both
emails_only_not_saiid = emails_not_saiid - emails_both

# 4️⃣ Counts
print("Count in both:", len(emails_both))
print("Count only in saiid:", len(emails_only_saiid))
print("Count only in non-saiid:", len(emails_only_not_saiid))

# Conditional logic
df['final_flag'] = (
    ((df['one_column'] == 1) & (df['min_date'] > df['application_date'])) |
    ((df['one_column'] == 0) & (df['application_date'] > fixed_date))
).astype(int)

In [ ]:
campaign_substr = 'hi'  # the substring to search for

# 1) Which rows are in a matching campaign?
mask = df['campaign'].astype(str).str.contains(campaign_substr, case=False, na=False)

# 2) All emails that EVER had a matching campaign
emails_with_match = set(df.loc[mask, 'email'])

# 3) Flag every row for those emails (1 if the email ever used the campaign, else 0)
df['ever_hi'] = df['email'].isin(emails_with_match).astype(int)

# 4) First date each email was associated with a matching campaign
first_hi_per_email = (
    df.loc[mask]
      .groupby('email', as_index=False)['date']
      .min()
      .set_index('email')['date']
)

# Map to every row; emails without a match get NaT
df['first_hi_date'] = df['email'].map(first_hi_per_email)

# (optional) If you'd rather fill non-matching emails with a sentinel string:
# df['first_hi_date'] = df['first_hi_date'].dt.strftime('%Y-%m-%d').fillna('never')

In [ ]:
# Group by score and calculate average sales rate
avg_sales = df.groupby("score")["actual"].mean().reset_index()

# Plot
plt.figure(figsize=(7,5))
plt.plot(avg_sales["score"], avg_sales["actual"], marker="o", linewidth=2)

# Style
plt.title("Average Sales Rate by Model Score", fontsize=14)
plt.xlabel("Model Score (1 = high sale likelihood, 5 = low)", fontsize=12)
plt.ylabel("Average Sales Rate", fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(avg_sales["score"])

# Annotate values
for x, y in zip(avg_sales["score"], avg_sales["actual"]):
    plt.text(x, y + 0.01, f"{y:.2%}", ha="center", fontsize=10)


In [ ]:
target_prefixes = ("1", "2", "3")  # numbers you want to match at the start

for json_file in json_files:
    try:
        obj = s3.get_object(Bucket=bucket_name, Key=json_file)
        file_content = obj['Body'].read().decode('utf-8')

        try:
            json_data = json.loads(file_content)
        except json.JSONDecodeError:
            print(f"File {json_file} is not valid JSON. Skipping.")
            continue

        if isinstance(json_data, dict):
            json_data = [json_data]
        elif not isinstance(json_data, list):
            print(f"Unexpected JSON format in {json_file}. Skipping.")
            continue

        # Filter JSON objects by key values starting with target numbers
        filtered_data = [
            item for item in json_data
            if any(str(v).startswith(target_prefixes) for v in item.values())
        ]

        all_data.extend(filtered_data)

    except Exception as e:
        print(f"Error processing file {json_file}: {e}")
        continue


In [ ]:
import boto3
import ijson
import orjson
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from botocore.config import Config
from io import BytesIO

# ==== CONFIG ====
bucket_name = 'your-bucket-name'
prefix = 'your-folder-name/'      # keep trailing slash
max_workers = 12                  # adjust to your network/CPU
max_pool_connections = 48
region = None                     # e.g., "us-west-2" or None to use default
profile = None                    # e.g., "your-aws-profile" or None to use env/default
# ===============

cfg = Config(max_pool_connections=max_pool_connections,
             retries={'max_attempts': 8, 'mode': 'standard'})

session = (boto3.session.Session(profile_name=profile, region_name=region)
           if profile or region else boto3.session.Session())
s3 = session.client('s3', config=cfg)

def list_json_keys(bucket: str, pfx: str):
    paginator = s3.get_paginator('list_objects_v2')
    page_it = paginator.paginate(Bucket=bucket, Prefix=pfx)
    exts = ('.json', '.jsonl', '.ndjson')
    out = []
    for page in page_it:
        for obj in page.get('Contents', []):
            k = obj['Key']
            if k.endswith(exts):
                out.append(k)
    return out

def parse_json_body(body_bytes: bytes):
    """
    Return a list[dict] from body_bytes.
    Tries JSONL -> JSON array -> single dict -> double-encoded.
    """
    rows = []

    # --- Try JSON Lines first
    bio = BytesIO(body_bytes)
    saw_jsonl = False
    for raw in bio:
        line = raw.strip()
        if not line:
            continue
        saw_jsonl = True
        try:
            rec = orjson.loads(line)
            if isinstance(rec, dict):
                rows.append(rec)
            elif isinstance(rec, list):
                rows.extend([x for x in rec if isinstance(x, dict)])
            # ignore scalars
        except orjson.JSONDecodeError:
            saw_jsonl = False
            break
    if saw_jsonl and rows:
        return rows
    elif saw_jsonl:
        # looked like lines but none were dicts; fall through to whole parse
        pass

    # --- Try top-level array (streaming)
    bio.seek(0)
    try:
        for item in ijson.items(bio, 'item'):
            if isinstance(item, dict):
                rows.append(item)
        if rows:
            return rows
    except Exception:
        pass

    # --- Whole-file parse (dict or list, possibly double-encoded)
    try:
        whole = orjson.loads(body_bytes)
    except orjson.JSONDecodeError:
        # Sometimes the body is actually text; try decoding then parsing
        try:
            whole = orjson.loads(body_bytes.decode('utf-8', errors='ignore'))
        except orjson.JSONDecodeError:
            return []

    # Double-encoded JSON: first parse yields a str
    if isinstance(whole, str):
        try:
            whole = orjson.loads(whole)
        except orjson.JSONDecodeError:
            return []

    if isinstance(whole, dict):
        # Common container keys; if absent, treat as single record
        for k in ('items', 'records', 'data', 'rows', 'result'):
            v = whole.get(k)
            if isinstance(v, list):
                rows.extend([x for x in v if isinstance(x, dict)])
                break
        else:
            rows.append(whole)
    elif isinstance(whole, list):
        rows.extend([x for x in whole if isinstance(x, dict)])

    return rows

def process_key(key: str):
    try:
        obj = s3.get_object(Bucket=bucket_name, Key=key)
        body = obj['Body'].read()
        return parse_json_body(body)
    except Exception as e:
        print(f"[ERROR] {key}: {e}")
        return []

# ---- Run ----
keys = list_json_keys(bucket_name, prefix)
print(f"[INFO] Found {len(keys)} files under s3://{bucket_name}/{prefix}")

dfs = []
with ThreadPoolExecutor(max_workers=max_workers) as ex:
    futures = [ex.submit(process_key, k) for k in keys]
    for fut in as_completed(futures):
        rows = fut.result()
        if rows:
            dfs.append(pd.DataFrame.from_records(rows))

if dfs:
    df = pd.concat(dfs, ignore_index=True)
    print("[INFO] DataFrame created:", df.shape)
    print(df.head())
else:
    df = pd.DataFrame()
    print("[INFO] No JSON objects parsed into rows.")


In [ ]:
import boto3, gzip, shutil, os

def download_s3_csv(bucket, key, out_dir='.', aws_access_key_id=None, aws_secret_access_key=None, aws_session_token=None, region_name=None):
    s3 = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        aws_session_token=aws_session_token,
        region_name=region_name
    )

    gz_path = os.path.join(out_dir, os.path.basename(key))
    csv_path = gz_path[:-3]

    s3.download_file(bucket, key, gz_path)
    with gzip.open(gz_path, 'rb') as f_in, open(csv_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    return csv_path


In [ ]:
🥇 1. Word Frequency Counter

Prompt:
Given a sentence (string), return a dictionary mapping each word to the number of times it appears.
Ignore capitalization and punctuation (.,!?).


Test cases:

print(word_frequency("This is a test. This test is simple."))
# {'this': 2, 'is': 2, 'a': 1, 'test': 2, 'simple': 1}

print(word_frequency("Hello hello world!"))
# {'hello': 2, 'world': 1}

🥈 2. Invert a Dictionary

Prompt:
Write a function that takes a dictionary d and returns a new dictionary where each value becomes a key, and each key is added to a list of keys that shared that value.

Test cases:

print(invert_dict({"a":1, "b":2, "c":1}))
# {1: ['a', 'c'], 2: ['b']}

print(invert_dict({"x":3, "y":3, "z":4}))
# {3: ['x', 'y'], 4: ['z']}

🥉 3. Character Counter

Prompt:
Given a string s, return a dictionary mapping each letter (case-insensitive) to the number of times it appears.
Ignore non-alphabetical characters.


Test cases:

print(char_count("Hello World!"))
# {'h': 1, 'e': 1, 'l': 3, 'o': 2, 'w': 1, 'r': 1, 'd': 1}

print(char_count("AaBbCc"))
# {'a': 2, 'b': 2, 'c': 2}

🏅 4. Group by First Letter

Prompt:
Given a list of words, group them by their first letter (case-sensitive).
Return a dictionary where keys are letters and values are lists of words starting with that letter.

Test cases:

print(group_by_first_letter(["apple","ant","banana","bat","cat"]))
# {'a': ['apple', 'ant'], 'b': ['banana', 'bat'], 'c': ['cat']}

print(group_by_first_letter(["dog", "deer", "cat"]))
# {'d': ['dog', 'deer'], 'c': ['cat']}

🏆 5. Merge Two Dictionaries with Summed Values

Prompt:
You are given two dictionaries with numeric values.
Merge them so that if a key exists in both, their values are summed.
Return a new dictionary.


Test cases:

print(merge_sum({"a":1,"b":2}, {"b":3,"c":4}))
# {'a': 1, 'b': 5, 'c': 4}

print(merge_sum({}, {"x":10}))
# {'x': 10}

In [ ]:
import boto3
import io
import gzip
import pandas as pd

def build_df_from_gzipped_s3_folder(
    bucket: str,
    prefix: str,
    header_first_token: str = "rampid",
    max_keys_per_page: int = 1000,
) -> pd.DataFrame:
    """
    Streams .gz files from s3://{bucket}/{prefix}, finds the header file whose first
    non-empty line starts with `header_first_token` (e.g., 'rampid'), uses that as columns,
    and concatenates all other files as data rows. No local writes.
    """
    s3 = boto3.client("s3")

    # 1) List all objects under the prefix
    paginator = s3.get_paginator("list_objects_v2")
    page_iter = paginator.paginate(Bucket=bucket, Prefix=prefix, PaginationConfig={"PageSize": max_keys_per_page})

    keys = []
    for page in page_iter:
        for obj in page.get("Contents", []):
            # Skip "folders" and non-gz files if present
            key = obj["Key"]
            if key.endswith("/") or not key.lower().endswith(".gz"):
                continue
            keys.append(key)

    if not keys:
        raise FileNotFoundError(f"No .gz objects found under s3://{bucket}/{prefix}")

    # 2) Helper to read first non-empty line from a gzipped S3 object
    def _first_nonempty_line(bkt, ky) -> str:
        body = s3.get_object(Bucket=bkt, Key=ky)["Body"]
        with gzip.GzipFile(fileobj=body) as gz:
            with io.TextIOWrapper(gz, encoding="utf-8", newline="") as fh:
                for line in fh:
                    stripped = line.strip()
                    if stripped:
                        return stripped
        return ""  # empty file

    # 3) Find header file: first non-empty line begins with 'rampid'
    header_key = None
    header_line = None
    header_token_lower = header_first_token.lower()

    for key in keys:
        first = _first_nonempty_line(bucket, key)
        if first and first.split(",")[0].strip().lower().startswith(header_token_lower):
            header_key = key
            header_line = first
            break

    if header_key is None or not header_line:
        raise ValueError(
            f"Could not find a header file whose first non-empty line starts with '{header_first_token}'."
        )

    # Turn the header line into a list of column names
    columns = [c.strip() for c in header_line.split(",")]

    # 4) Stream and parse all other files as rows (also tolerate if some files repeat header)
    dataframes = []
    for key in keys:
        if key == header_key:
            # We already used its first line as header; skip the rest of this file
            # (If you actually need to include its data rows below the header line,
            # flip this to read the file and skip the first row during parsing.)
            continue

        body = s3.get_object(Bucket=bucket, Key=key)["Body"]

        # Wrap gzip->text stream for pandas
        with gzip.GzipFile(fileobj=body) as gz:
            # We will read with header=None and assign names=columns.
            # Then, drop any accidental header rows where first col == 'rampid'
            df = pd.read_csv(
                io.TextIOWrapper(gz, encoding="utf-8", newline=""),
                header=None,
                names=columns,
                sep=",",
                engine="python",
                dtype=str,      # safer for mixed/ID-like fields; cast later as needed
            )

            # Drop accidental header rows present in data files
            first_col = df.columns[0]
            df = df[df[first_col].str.lower() != header_token_lower]

            if not df.empty:
                dataframes.append(df)

    if not dataframes:
        # It's possible all rows were in the header file. If you want to include
        # data rows from the header file (after the header row), you can read it too.
        return pd.DataFrame(columns=columns)

    # 5) Concatenate
    out = pd.concat(dataframes, ignore_index=True)

    return out


# ---- Example usage ----
# df = build_df_from_gzipped_s3_folder(
#     bucket="my-bucket",
#     prefix="path/to/folder/",
#     header_first_token="rampid"
# )
# print(df.head())
